In [2]:
import pandas as pd
from folium.plugins import HeatMap
import plotly.express as px
import folium
import matplotlib.pyplot as plt
import geopy
from geopy.geocoders import Nominatim
import json
from shapely.geometry import shape
from shapely.prepared import prep
from shapely.geometry import Point

In [3]:
df = pd.read_csv(r'C:\Users\KOSTAS\Desktop\projects\E_D_A-projects-python\Earthquakes_Analysis\data\EarthQuakes in Greece.csv')

In [4]:
df.shape

(256655, 8)

In [5]:
df.head()

,Year,Month,Date,Hours,Minutes,LATATITUDE (N),LONGITUDE (E),MAGNITUDE (Richter)
0,1901,9,12,6,15,39.0,22.2,5.6
1,1901,10,25,16,18,37.0,22.2,5.4
2,1901,12,24,23,18,37.2,22.2,5.8
3,1902,4,11,18,35,38.5,23.5,5.8
4,1902,7,5,14,56,40.8,23.2,6.6


In [6]:
df.isnull().sum()

Year                   0
Month                  0
Date                   0
Hours                  0
Minutes                0
LATATITUDE (N)         0
LONGITUDE  (E)         0
MAGNITUDE (Richter)    0
dtype: int64

In [7]:
df.rename({'LATATITUDE (N)': 'Latitude', 'LONGITUDE  (E)': 'Longitude', 'MAGNITUDE (Richter)': 'Magnitude (Richter)'}, 
      axis=1, inplace=True)

In [8]:
df['coordinates'] = list(zip(df['Latitude'],df['Longitude']))

### Exclude coordinates that do not belong in Greek territory

In [11]:
with open(r'C:\Users\KOSTAS\Desktop\projects\E_D_A-projects-python\Earthquakes_Analysis\data\Greece_AL2.GeoJson', 'rb') as json_file:
    data = json.load(json_file)

In [12]:
data

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'srid': '4326',
    'id': '192307',
    'name': 'Greece',
    'localname': 'Ελλάδα',
    'official_name': 'Ελληνική Δημοκρατία',
    'boundary': 'administrative',
    'admin_level': '2',
    'note': '',
    'wikidata': 'Q41',
    'wikipedia': 'el:Ελλάδα',
    'timestamp': '2019-10-06 22:02:01',
    'rpath': '192307,0',
    'alltags': {'flag': 'http://upload.wikimedia.org/wikipedia/commons/5/5c/Flag_of_Greece.svg',
     'name': 'Ελλάδα',
     'name:ab': 'Барзентәыла',
     'name:af': 'Griekeland',
     'name:am': 'ግሪክ',
     'name:an': 'Grecia',
     'name:ar': 'اليونان',
     'name:ay': 'Grisya',
     'name:az': 'Yunanıstan',
     'name:ba': 'Греция',
     'name:be': 'Грэцыя',
     'name:bg': 'Гърция',
     'name:bi': 'Greece',
     'name:bn': 'গ্রিস',
     'name:bo': 'ཀེ་རི་སི།',
     'name:br': 'Gres',
     'name:bs': 'Grčka',
     'name:ca': 'Grècia',
     'name:ce': 'Греци',
     'name:co': 'Grecia',


In [13]:
countries = {}
gr = '' # multipolygon
for feature in data['features']:
    geom = feature['geometry']
    gr = shape(geom)
    countries['Greece'] = prep(gr)
# stores greek coordinates

In [20]:
def get_country(row):
    point = Point(row['Longitude'], row['Latitude'])
    for country, geom in countries.items():
        if geom.contains(point):
            return True # country name
    return False # unknown

In [21]:
df['Country'] = df.apply(get_country,axis=1)

In [22]:
df.head()

,Year,Month,Date,Hours,Minutes,Latitude,Longitude,Magnitude (Richter),coordinates,Country
0,1901,9,12,6,15,39.0,22.2,5.6,"(39.0, 22.2)",True
1,1901,10,25,16,18,37.0,22.2,5.4,"(37.0, 22.2)",True
2,1901,12,24,23,18,37.2,22.2,5.8,"(37.2, 22.2)",True
3,1902,4,11,18,35,38.5,23.5,5.8,"(38.5, 23.5)",True
4,1902,7,5,14,56,40.8,23.2,6.6,"(40.8, 23.2)",True


In [23]:
df[df['Country'] == False].shape # 59596 coordinates do not belong in Greece

(59596, 10)

In [24]:
df = df[df['Country'] == True]

In [25]:
df.shape

(197059, 10)

# Make a dataframe using earthquakes of at least 5 Richter

In [26]:
major_df = df[df['Magnitude (Richter)'] >= 5]

#### Find district of places for each major earthquake

In [27]:
major_df['lat'] = list(map(str,major_df['Latitude']))
major_df['long'] = list(map(str,major_df['Longitude']))

C:\Users\KOSTAS\AppData\Local\Temp/ipykernel_14056/144862577.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  major_df['lat'] = list(map(str,major_df['Latitude']))
C:\Users\KOSTAS\AppData\Local\Temp/ipykernel_14056/144862577.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  major_df['long'] = list(map(str,major_df['Longitude']))


In [28]:
def coord(a,b):
    return a + " " + b

In [29]:
major_df['coord'] = list(map(coord,major_df['lat'],major_df['long']))

C:\Users\KOSTAS\AppData\Local\Temp/ipykernel_14056/2113947172.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  major_df['coord'] = list(map(coord,major_df['lat'],major_df['long']))


In [30]:
def get_loc(coordinates):
    """
    This function will return information about the place, given the coordinates. If there is no such district, it will return
    that the center of earthquake was in the sea
    """
    try:
        locator = Nominatim(user_agent="myGeocoder")
        location = locator.reverse(coordinates)
        return location.raw['address']['state_district'] 
    except:
        return "Sea"

In [31]:
major_df['district'] = list(map(get_loc,major_df['coord']))

C:\Users\KOSTAS\AppData\Local\Temp/ipykernel_14056/853323575.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  major_df['district'] = list(map(get_loc,major_df['coord']))


#### Drop columns that I will not need any more

In [32]:
major_df.drop(['lat','long','coord'], axis = 1, inplace = True)

C:\Users\KOSTAS\AppData\Local\Temp/ipykernel_14056/2113591972.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  major_df.drop(['lat','long','coord'], axis = 1, inplace = True)


In [33]:
major_df.head()

,Year,Month,Date,Hours,Minutes,Latitude,Longitude,Magnitude (Richter),coordinates,Country,district
0,1901,9,12,6,15,39.0,22.2,5.6,"(39.0, 22.2)",True,Περιφέρεια Στερεάς Ελλάδος
1,1901,10,25,16,18,37.0,22.2,5.4,"(37.0, 22.2)",True,Περιφέρεια Πελοποννήσου
2,1901,12,24,23,18,37.2,22.2,5.8,"(37.2, 22.2)",True,Περιφέρεια Πελοποννήσου
3,1902,4,11,18,35,38.5,23.5,5.8,"(38.5, 23.5)",True,Περιφέρεια Στερεάς Ελλάδος
4,1902,7,5,14,56,40.8,23.2,6.6,"(40.8, 23.2)",True,Περιφέρεια Κεντρικής Μακεδονίας


#### Translate the district column

In [34]:
major_df['district'].unique()

array(['Περιφέρεια Στερεάς Ελλάδος', 'Περιφέρεια Πελοποννήσου',
       'Περιφέρεια Κεντρικής Μακεδονίας', 'Περιφέρεια Δυτικής Ελλάδας',
       'Περιφέρεια Ιονίων Νήσων', 'Sea', 'Περιφέρεια Αττικής',
       'Περιφέρεια Βόρειου Αιγαίου', 'Περιφέρεια Θεσσαλίας',
       'Περιφέρεια Ανατολικής Μακεδονίας και Θράκης', 'Περιφέρεια Κρήτης',
       'Περιφέρεια Νοτίου Αιγαίου', 'Περιφέρεια Δυτικής Μακεδονίας',
       'Περιφέρεια Ηπείρου'], dtype=object)

In [35]:
major_df['district'].replace({'Περιφέρεια Στερεάς Ελλάδος': 'Central Greece', 'Περιφέρεια Πελοποννήσου': 'Peloponnese',
                              'Περιφέρεια Κεντρικής Μακεδονίας': 'Central Macedonia', 'Περιφέρεια Δυτικής Ελλάδας': 'Western Greece',
                              'Περιφέρεια Ιονίων Νήσων':'Ionian Islands', 'Sea': 'Mediterranean Sea', 
                              'Περιφέρεια Αττικής': 'Attica', 'Περιφέρεια Βόρειου Αιγαίου': 'North Aegean', 
                              'Περιφέρεια Θεσσαλίας': 'Thessaly', 'Περιφέρεια Ανατολικής Μακεδονίας και Θράκης':'Thrace',
                              'Περιφέρεια Κρήτης': 'Crete', 'Περιφέρεια Νοτίου Αιγαίου': 'South Aegean', 
                              'Περιφέρεια Δυτικής Μακεδονίας': 'Western Macedonia',
                              'Περιφέρεια Ηπείρου': 'Epirus'}, inplace=True)

C:\Users\KOSTAS\AppData\Local\Temp/ipykernel_14056/2751764813.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  major_df['district'].replace({'Περιφέρεια Στερεάς Ελλάδος': 'Central Greece', 'Περιφέρεια Πελοποννήσου': 'Peloponnese',


In [36]:
major_df.rename({'district': 'District'}, axis=1, inplace=True)

C:\Users\KOSTAS\AppData\Local\Temp/ipykernel_14056/3996156017.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  major_df.rename({'district': 'District'}, axis=1, inplace=True)


#### Export the major dataframe to csv

In [37]:
major_df.to_csv('major_earthquake_data.csv', index=False)

# Initial dataframe (with all earthquakes)

#### Create a column to classify an earthquake depending on its magnitude

In [38]:
df.head()

,Year,Month,Date,Hours,Minutes,Latitude,Longitude,Magnitude (Richter),coordinates,Country
0,1901,9,12,6,15,39.0,22.2,5.6,"(39.0, 22.2)",True
1,1901,10,25,16,18,37.0,22.2,5.4,"(37.0, 22.2)",True
2,1901,12,24,23,18,37.2,22.2,5.8,"(37.2, 22.2)",True
3,1902,4,11,18,35,38.5,23.5,5.8,"(38.5, 23.5)",True
4,1902,7,5,14,56,40.8,23.2,6.6,"(40.8, 23.2)",True


In [39]:
df['Magnitude (Richter)'].max()

8.0

In [40]:
def earthquake_class(magn):
    if magn < 2:
        return "Micro_Earthquake"
    elif 2 <= magn < 4:
        return "Minor_Earthquake"
    elif 4 <= magn < 5:
        return "Light_Earthquake"
    elif 5 <= magn <6:
        return "Moderate_Earthquake"
    elif 6 <= magn < 7:
        return "Strong_Earthquake"
    elif 7 <= magn <= 8:
        return "Great_Earthquake"

In [41]:
df['Earthquake_class'] = list(map(earthquake_class,df['Magnitude (Richter)']))

In [42]:
df.head()

,Year,Month,Date,Hours,Minutes,Latitude,Longitude,Magnitude (Richter),coordinates,Country,Earthquake_class
0,1901,9,12,6,15,39.0,22.2,5.6,"(39.0, 22.2)",True,Moderate_Earthquake
1,1901,10,25,16,18,37.0,22.2,5.4,"(37.0, 22.2)",True,Moderate_Earthquake
2,1901,12,24,23,18,37.2,22.2,5.8,"(37.2, 22.2)",True,Moderate_Earthquake
3,1902,4,11,18,35,38.5,23.5,5.8,"(38.5, 23.5)",True,Moderate_Earthquake
4,1902,7,5,14,56,40.8,23.2,6.6,"(40.8, 23.2)",True,Strong_Earthquake


#### Export dataset to csv 

In [43]:
df.to_csv('earthquake_data.csv', index=False)